<a href="https://colab.research.google.com/github/gyuwonchoi/PytorchTutorial/blob/main/QuickStart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets                # especially for vision 
from torchvision.transforms import ToTensor

In [43]:
training_data=datasets.FashionMNIST(            # module from torchvision
    root='data',                                # download path 
    train=True,                                 # differs the data set (train-images-idx3-ubyte, otherwise from t10k-images-idx3-ubyte.) : later one for test 
    download=True,                              # whether download or not
    transform=ToTensor()
)

In [44]:
test_data=datasets.FashionMNIST(            # module from torchvision
    root='data',                            # download path 
    train=False,                            # t10k-images-idx3-ubyte : later one for test 
    download=True,                          # whether download or not
    transform=ToTensor()
)

In [45]:
batch_size = 64

# create dataloader from torch.utils.data module
train_dataloader = DataLoader(training_data, batch_size=batch_size)  # dataset => mini batch 
test_dataloader= DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")                     # f for fromatting 
    print(f"Shape of y: {y.shape}/{y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])/torch.int64


In [46]:
# check GPU
device ="cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# define model
class NeuralNetwork(nn.Module):        # nn module from torch 
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()        # Replace nn.Flattne() format to self.flatten: use when you make new instance
                                       # Instantiating a class: creating a copy of the class which inherits all class variables and methods.
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),         # not 28,28 but 784
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)             # 10 classes , error code: out_features: used . (should be ,)
    )
  
  def forward(self, x):
      x = self.flatten(x)             
      logits= self.linear_relu_stack(x)
      return logits

model= NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Loss Function and Optimizer

In [47]:
loss_func = nn.CrossEntropyLoss()  # softmax + cross entropy : you dont need to add softmax at the end of model
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

##  Training 

In [48]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)      #  total training data size 

  test_loss, correct = 0,0

  for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)   # copy all the tensor variables at the beginning of reading data to the GPU
                                          # declare variable also
        
  # error from predictec result 
      pred = model(X)
      loss = loss_func(pred, y)
     
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if batch % 100 == 0:
        loss, current = loss.item(), batch*len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [51]:
def test(dataloader, model, loss_fn):
  size=len(dataloader.dataset)
  num_batches= len(dataloader)
  model.eval()

  test_loss, correct =0,0

  with torch.no_grad(): # because it is test time
    for X, y in dataloader: # in batch 
        X, y = X.to(device), y.to(device) # to GPU
        pred=model(X)   # forward propa
 
        test_loss+= loss_func(pred,y).item() # if reduction is true of nn.CrossEntropyLoss(), its output is scalar or same as input 
                                             # loss btw 2 probabilities 
        correct+= (pred.argmax(1) == y).type(torch.float).sum().item() # y is G.T, 
                                                                       # output : prob of each class : vector
        #print((pred.argmax(1) == y).type(torch.float).sum()) # every 10 scores of each img 
                                                              # dimen 1 for row-wise 

  test_loss /= num_batches
  correct /= size

  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [52]:
epoch = 5

for t in range(5):
    train(train_dataloader, model, loss_func, optimizer)
    test(test_dataloader, model, loss_func)
  
print("done!")

loss: 1.153190  [    0/60000]
loss: 1.149499  [ 6400/60000]
loss: 0.975585  [12800/60000]
loss: 1.100361  [19200/60000]
loss: 0.981066  [25600/60000]
loss: 1.014883  [32000/60000]
loss: 1.047661  [38400/60000]
loss: 0.992606  [44800/60000]
loss: 1.026445  [51200/60000]
loss: 0.950011  [57600/60000]
Test Error: 
 Accuracy: 65.7%, Avg loss: 0.972744 

loss: 1.037337  [    0/60000]
loss: 1.054595  [ 6400/60000]
loss: 0.864105  [12800/60000]
loss: 1.011716  [19200/60000]
loss: 0.898545  [25600/60000]
loss: 0.923932  [32000/60000]
loss: 0.973794  [38400/60000]
loss: 0.921687  [44800/60000]
loss: 0.950693  [51200/60000]
loss: 0.885989  [57600/60000]
Test Error: 
 Accuracy: 67.2%, Avg loss: 0.903523 

loss: 0.953239  [    0/60000]
loss: 0.989826  [ 6400/60000]
loss: 0.785194  [12800/60000]
loss: 0.949126  [19200/60000]
loss: 0.843428  [25600/60000]
loss: 0.857911  [32000/60000]
loss: 0.921763  [38400/60000]
loss: 0.874444  [44800/60000]
loss: 0.896027  [51200/60000]
loss: 0.839938  [57600/600

## 모델 Load 및 Save


In [ ]:
model_path = "quick_start_pth"
torch.save(model.state_dict(), model_path) # saved object: state_dict => , file path 
print(f"saved to {model_path}")  # save process?

saved to quick_start_pth


In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()  # turn mode to test time 
x, y = test_data[0][0], test_data[0][1] # (image, target) where target is index of the target class

with torch.no_grad():
    pred = model(x)
    predicted, actual=classes[pred[0].argmax(0)], classes[y] # see above classes table : QUESTION // constant for dimension 0 for vector ways

    print(pred[0])
    print("==========================")

    print(f'Predicted: "{predicted}", Actual: "{actual}"')

tensor([-2.4465, -2.6904, -1.0338, -1.9402, -0.9676,  2.4919, -1.2347,  2.7351,
         1.8703,  3.1323])
Predicted: "Ankle boot", Actual: "Ankle boot"
